# Feature Extraction and Unsupervised Learning with Tensorboard
This notebook extracts features from a directory of midi files using Music21's jSymbolic feature extractors. This is a compute-heavy task and can take between 10-30 seconds per midi file. Once extraction has been performed, the results can be cached (using pickle) and dimensionality reduction/visualization can be done using tensorboard's embedding projector.

## Imports

In [2]:
import time, os, csv, json, math, sys
sys.path.append('../python')

import utils
import dill as pickle
from multiprocessing import Pool as ThreadPool
import numpy as np
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

## Extract Features
It is recommended to run this function sparingly and cache the results with pickle below to refrain from unecessary compute.

In [ ]:
symlink_dir = '../../data/query_symlinks'

num_threads = 8
pool = ThreadPool(num_threads)

start_time = time.time()
extracted_features = pool.map(utils.extract_features, [os.path.join(symlink_dir, n) for n in os.listdir(symlink_dir)])
print('Finished in {:.2f} seconds'.format(time.time() - start_time))

## Save/Load cached feature extraction

In [3]:
## save cache
# with open('../../data/extracted_features/4000_midi_files_features.pickle', 'w') as f:
#     pickle.dump(extracted_features, f)

## load cache
with open('../../data/extracted_features/4000_midi_files_features.pickle', 'r') as f:
    extracted_features = pickle.load(f)

## Load MSD Cache

In [4]:
start_time = time.time()
with open('../../data/msd.pickle', 'r') as f:
    msd = pickle.load(f)
print('Loaded in {:.2f} seconds'.format(time.time() - start_time))

Loaded in 10.63 seconds


## Analyze and remove unnecessary features
Clean up the results from the music21 feature extraction.

In [5]:
vec = []
for f in extracted_features:
    if f is not None:
        arr = f[0][0]
        arr.pop(0) # first element is an empty string
        arr.pop(-1) # last element is an empty string
        vec.append(np.array(arr))
embeddings = np.asarray(vec)

In [ ]:
# summed = np.sum(embeddings, axis=0)
# for i, n in enumerate(extracted_features[0][1][1:-1]):
#     print('{}:     {}'.format(n, summed[i]))

## Tensorboard Labels/Metadata
Create a `metadata.tsv` file to assosciate msd track metadata to data points in tensorboard.

In [7]:
LOG_DIR='../../data/embedding_logdir'
with open(os.path.expanduser('~') + '/Documents/code/midi-dataset/data/match_scores.json', 'r') as f:
    match_scores = json.load(f)

In [8]:
def get_mid_to_track(match_scores):
    mid_to_track = {}
    for k, vals in match_scores.items():
        for v in vals:
            if not v in mid_to_track:
                mid_to_track[v] = {
                    'track': k,
                    'confidence': match_scores[k][v]
                }
            # if the new confidence score is higher than the last
            # update the track it points to
            elif match_scores[k][v] > mid_to_track[v]['confidence']:
                mid_to_track[v]['track'] = k
    return mid_to_track

def get_track_to_msd(msd):
    return { m['track_id']: m for m in msd }

def save_features_to_tsv(filename, features, mid_to_track, track_to_msd):
    with open(filename, 'w') as csvfile:
        
        fieldnames = ['path', 
                      'song_year',
                      'song_title',
                      'song_time_signature',
                      'song_tempo',
                      'song_key',
                      'song_mode',
                      'song_loudness',
                      'song_energy',
                      'song_duration',
                      'song_danceability',
                      'song_hotttnesss',
                      'artist_name',
                      'artist_terms',
                      'artist_mbtags',
                      'artist_hotttnesss',
                      'artist_location']

        writer = csv.DictWriter(csvfile, delimiter='\t', fieldnames=fieldnames)

        writer.writeheader()
        for f in extracted_features:
            if f is not None:
                path = f[1][-1]
                basename = os.path.basename(path)[0:-4]
                if basename in mid_to_track:
                    track = mid_to_track[basename]['track']
                    d = {k: v for k, v in track_to_msd[track].items() if k in fieldnames}
                    for k, v in d.items():
                        if isinstance(v, np.ndarray):
                            d[k] = v.tolist()[0] if len(v.tolist()) > 0 else ''#','.join(v.tolist())
                        try:
                            f = float(v)
                            if math.isnan(v):
                                d[k] = 0.0
                        except:
                            pass
                    writer.writerow(d)                     

mid_to_track = utils.get_midi_to_track_lut() #get_mid_to_track(match_scores)
track_to_msd = get_track_to_msd(msd)
tsv = save_features_to_tsv(os.path.join(LOG_DIR, 'metadata.tsv'), extracted_features, mid_to_track, track_to_msd)

## Tensorboard Embeddings
Expose the embeddings to be used tensorboard.

In [ ]:
sess = tf.Session()

# create embeddings var
emb = tf.Variable(embeddings, name='embeddings')

# embedding projector
summary_writer = tf.train.SummaryWriter(LOG_DIR)
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = emb.name
embedding.metadata_path = os.path.join(LOG_DIR, 'metadata.tsv')
projector.visualize_embeddings(summary_writer, config)

# init and run the session
init = tf.global_variables_initializer()
sess.run(init)

# save checkpoint
saver = tf.train.Saver()
saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), 0)